In [1]:
import selenium
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import WebDriverException, TimeoutException
from tqdm import tqdm
import time

In [12]:

class Scraper:
#to access website
#initializing variables
    def __init__(self):
        self.driver = webdriver.Chrome(ChromeDriverManager().install())
        url = "https://www.zoopla.co.uk/new-homes/property/london/?q=London&results_sort=newest_listings&search_source=new-homes&page_size=25&pn=1&view_type=list"
        self.driver.get(url)
        time.sleep(2)
        self.delay = 10
        self.big_list = []

    #to click the cookies box
    def load_and_accept_cookies(self):
        try:
            WebDriverWait(self.driver, self.delay).until(EC.presence_of_element_located((By.XPATH, '//*[@id="gdpr-consent-notice"]')))
            self.driver.switch_to.frame('gdpr-consent-notice')
            accept_cookies_button = WebDriverWait(self.driver, self.delay).until(EC.presence_of_element_located((By.XPATH, '//*[@id="save"]')))
            accept_cookies_button.click()
            time.sleep(1)
        except TimeoutException:
            print("Loading took too much time!")
        return self.driver
        
    #to get the links on each page
    def get_links(self):
        links = []
        time.sleep(5)
        link_list = self.driver.find_element(By.XPATH, '//div[@class="css-1itfubx e34pn540"]')
        list = link_list.find_elements(By.XPATH, './div')
        for sites in list:
            link_tag = sites.find_element(By.TAG_NAME, 'a')
            links.append(link_tag.get_property('href'))
        print(f"There are {len(links)} properties on this page")
        return links

    #to click to go into the next page
    def next_page(self):
        delay = 10
        for i in range(5):
            self.big_list.extend(self.get_links())
            time.sleep(10)
            page = self.driver.find_element(By.XPATH, '//li[@class="css-qhg1xn-PaginationItemPreviousAndNext-PaginationItemNext eaoxhri2"]')
            page.click()
            try:
                WebDriverWait(self.driver, 3).until(EC.presence_of_element_located((By.XPATH, '//div[@class="css-1vc84cj-CancelButtonWrapper e13xjwxo5"]/button'))).click()
                print("Frame Ready!")
            except:
                pass
        print(f"There are {len(self.big_list)} properties on this pages")
        return self.big_list

    # to get data from each house and put into a dictionary
    def get_data(self):
        self.next_page()
        for link in self.big_list:
            try:
                self.driver.get(link)
            except:
                print('link not found')
                dict_properties = {'Price': [], 'Address': [], 'Bedrooms': [], 'Description': []}
                price = self.driver.find_element(By.XPATH, '//div[@class="c-PJLV c-PJLV-igTumRt-css"]/p').text
               
                dict_properties['Price'].append(price)
                address = self.driver.find_element(By.XPATH, '//address[@data-testid="address-label"]').text
                # address
                dict_properties['Address'].append(address)
                bedrooms = self.driver.find_element(By.XPATH, '//div[@class="c-PJLV c-PJLV-ieHhfWi-css"]').text
                # bedrooms
                dict_properties['Bedrooms'].append(bedrooms)
                # descriptions
                div_tag = self.driver.find_element(By.XPATH, '//div[@class="css-14s800p e126pxpr3"]/span')
                description = div_tag.text
                dict_properties['Description'] = description
                print(dict_properties)

        self.driver.quit()

bot = Scraper()
load = bot.load_and_accept_cookies()

bot.get_data()



[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 103.0.5060
[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome
[WDM] - Driver [/Users/pearl/.wdm/drivers/chromedriver/mac64_m1/103.0.5060.53/chromedriver] found in cache
/var/folders/k4/lsf7875j0pd9cxljb33jtqb40000gn/T/ipykernel_4334/180015478.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  self.driver = webdriver.Chrome(ChromeDriverManager().install())


There are 25 properties on this page
Frame Ready!
There are 25 properties on this page
There are 25 properties on this page
There are 25 properties on this page
There are 25 properties on this page
There are 125 properties on this pages
link not found
{'Price': ['£435,000'], 'Address': ['Croham Valley Road, South Croydon, Surrey CR2'], 'Bedrooms': ['2 bed flat for sale'], 'Description': "**Meet our mortgage advisor to discuss the advantages of purchasing at Croham Valley Road. Reserve before 31st August to receive a gift voucher up to the value of £3000 when you move in!** Available with a 40% Help to Buy: Equity Loan for first time buyers!\n\nThis property is a luxury development of nine apartments set along one of South Croydon's most desirable streets. Plot 3 is a 3 bedroom ground floor apartment. All homes benefit from either two or three bedrooms, an en-suite to the principal bedroom and an open plan living area. Each property features a contemporary kitchen with integrated applia

MaxRetryError: HTTPConnectionPool(host='localhost', port=52985): Max retries exceeded with url: /session/9613c8a471478c00ce6b4e8bd0b2f40f/element (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x119b93cd0>: Failed to establish a new connection: [Errno 61] Connection refused'))